In [1]:
#Current edition notes: Fixes for plot pop-up, collapsible frames, implemented
#Next steps: Upload button/function for versioning
#Pads style cursor with scan, scan window width, IO sliders, analysis region sliders reimplemented
#Automatically remove "reserved columns"
#Default plot scattering, absorbing when doing respective calibration, set default values for entry fields
#Reimplement I0 selection
#Concatenate multiple files?
#SigFigs on r2 report
#plot arbitrary traces against each other

In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  
NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler

import tkinter as tk
import tkinter.ttk as ttk
from tkinter import filedialog
from datetime import datetime

from PIL import ImageTk, Image
import seaborn as sns
from scipy.stats import linregress
import warnings


import os
import sys


In [ ]:
window = tk.Tk()
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()
window.title("PAX Data Visualizer - Plotting in TKinter")
window.geometry(str(int(screen_width*.8))+"x"+str(int(screen_height*.8)))

#ToDo: Set screen_width ratio in constants.py

''

In [4]:
#An absolute file path function to help package images later using pyinstaller

def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)


In [5]:
# ax = [0]*8

col_index = 0

In [6]:
"""
This section is for the alarm code handling. Currently no implementation exists to try/except if it doesn't match. Manually check to make sure the names are current if having any issues.
"""


alarm_names = [
    "Bscat (1/Mm)",
    "scat_raw",
    "Babs (1/Mm)",
    "Babs phase (deg)",
    "Babs noise (1/Mm)",
    "Detected Laser power (W)",
    "Laser power phase (deg)",
    "Q factor",
    "Reserved",
    "Resonance Frequency (Hz)",
    "Background Bscat (1/Mm)",
    "mic_raw",
    "Background Babs (1/Mm)",
    "Background Babs phase (deg)",
    "Bext (1/Mm)",
    "Single Scat Albedo",
    "BC Mass (ug/m3)",
    "Relative Humidity (%)",
    "Cell Temperature (C)",
    "Dewpoint (C)",
    "Analong Input 1",
    "Analong Input 2",
    "Calibration Bscat",
    "Calibration Babs",
    "Reserved",
    "Reserved",
    "HK Case Temperature (C)",
    "HK Case Pressure (mbar)",
    "Reserved",
    "HK plus5V",
    "HK 3.3V",
    "Debug Ext Calculation",
    "HK 5Vdig",
    "Calibration I0",
    "Reserved",
    "Reserved",
    "HK Laser PD Current (Amp)",
    "HK Laser Current (Amp)",
    "HK Laser Temp (C)",
    "HK minus5V",
    "HK Cell Pressure (mbar)",
    "HK Inlet Pressure (mbar)",
    "HK Sample Pump Vac (mbar)",
    "HK 12V",
    "Reserved",
    "Mode",
    "Countdown Timer (secs)",
    "Disk Free Space (Gbytes)",
    "Laser on Time (hours)",
    "Reserved",
    "USB Status",
] 

In [7]:
#This is to ignore a deprecated functionality warning
warnings.filterwarnings("ignore", "use_inf_as_na")

In [8]:
#load file - the default for PAX happens to be CSV
def load_file():
	"""
	Loading default style PAX files. M0 files can technically be loaded, but functionality will be reduced.
	"""
	pb.start()
	global file_path
	if selected.get()=="V1":
		file_path = filedialog.askopenfilename(title="Choose PAX Data File", filetypes=(("Comma Separated","*.csv"),))
	elif selected.get()=="V2":
		file_path = filedialog.askopenfilename(title="Choose PAX Data File", filetypes=(("PAX Data","*.xlsx"),))
	else:
		tk.messagebox.showinfo("File Selection Error","This program does not currently suppot non xlsx or csv imports")

	pb.stop()
	pax_analyzer()

In [9]:
def pax_analyzer():
	"""
	Creating the pd df frames from files, cleaning/prepping the df
    Of note, recently changed the global "data" to "df" to prevent namespace overlap
	"""
    
	print("File path is "+file_path+".")
	global df

    #ToDo: Conditions with excel files, maybe a version to handle M0 files
	if selected.get() == "V1":
		df=pd.read_csv(file_path)
	if selected.get() == "V2":
		df=pd.read_excel(file_path)

    #NaN cleared prior to any datetime operations
	clearNaN()
    
	time = pd.to_datetime(df['Local Date'].astype(str) + ',' + df['Local Time'].astype(str), format = '%Y-%m-%d,%H:%M:%S')

    #ToDo: Give warning if column headers do not exactly match expected

    
	df.drop(columns = ['Sec UTC','DOY UTC','Year UTC','Sec Local','DOY Local','Year Local','Local Date', 'Local Time', 'Reserved.1', 'Reserved.2', 'Reserved.3', 'Reserved.4', 'Reserved.5'], inplace=True)
	df['time'] = time

    #List the various data columns to be selected later (would be off by one, but we move time to the end of the list)
	i = 0
	listbox.delete('0','end')
	for column in df.columns:
		if column == 'Alarm' or column == 'time':
			pass
		else:
			listbox.insert(i,column)
			if (i%2) == 0:
				listbox.itemconfigure(i, background = '#f0f0f0')
		i = i+1


In [10]:
#Creates a collapsible tkinter frame, for selectively hiding elements. Each instance of the collapsible frame can toggle itself
class CollapsibleFrame(ttk.Frame):
    def __init__(self, parent, title="", *args, **kwargs):
        super().__init__(parent, *args, **kwargs)

        self.title_frame = ttk.Frame(self)
        self.title_frame.pack(fill="x", expand=1)

        self.title_label = ttk.Label(self.title_frame, text=title)
        self.title_label.pack(side="left", fill="x", expand=1)

        self.toggle_button = ttk.Button(self.title_frame, text="▼", width=2, command=self.toggle)
        self.toggle_button.pack(side="right")

        self.sub_frame = ttk.Frame(self, relief="sunken", borderwidth=1)
        self.sub_frame.pack(fill="x", expand=1)

        self.is_collapsed = False

    def toggle(self):
        if self.is_collapsed:
            self.sub_frame.pack(fill="x", expand=1)
            self.toggle_button.config(text="▼")
        else:
            self.sub_frame.forget()
            self.toggle_button.config(text="▲")
        self.is_collapsed = not self.is_collapsed

In [11]:
def clearNaN():
	"""
	Small function to loop through the columns and clean up the df - consider making more complex? 
	"""
	for column in df.columns:
		df.replace([np.inf, -np.inf], np.nan, inplace=True)
		df.bfill()
		

In [12]:
#make figure as its own element
class FigureCreate:
    def __init__(self, figsize=(8, 6)):
        self.fig = plt.figure(figsize=figsize)
    
    def get_figure(self):
        return self.fig

In [13]:
#make axes based on listbox selection
class AxesCreate:
    def __init__(self, figure, position=(1, 1, 1)):
        self.ax = figure.add_subplot(*position)
    
    def get_axes(self):
        return self.ax

In [14]:
#plot a preselected group of elements
class CanvasDraw:
    def __init__(self, axes):
        self.ax = axes
    
    def plot(self, x, y, label=''):
        self.ax.plot(x, y, label=label)
    
    def show(self):
        self.ax.legend()
        plt.show()

In [ ]:
#main element of the view subsection; set up 

def plot(*args): 
    """
    THIS SECTION IS UNDER CONSTRUCTION. CURRENTLY TRYING TO TRANSITION TO SEABORN TO SEE IF IT IS MORE POWERFUL
    """
    #Close any extra plots so the graph is only presented in the one canvas
    plt.close()
    
    fig_create = FigureCreate(figsize=(5,5))
    fig = fig_create.get_figure()
    
    ax_create = AxesCreate(fig, position=(1,1,1))
    ax = ax_create.get_axes()
    
    selection = listbox.curselection()

    # These locations represent the position of sliders; because working with datetime is hard, we're breaking it down into 1000 units
    xloc1 = int(len(df['time']) * (current_value1.get() / 1000))
    xloc2 = int(len(df['time']) * (current_value2.get() / 1000))
    xlocA = int(len(df['time']) * (current_valueA.get() / 1000))
    xlocB = int(len(df['time']) * (current_valueB.get() / 1000))    


    
    for trace in selection:
        # ax.clear()
        locator = mdates.AutoDateLocator()
        formatter = mdates.ConciseDateFormatter(locator)
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)
        
        sns.lineplot(data=df, x='time', y=df.columns[trace], ax=ax)
        plt.xticks(rotation=20)

    
    plt.axvline(df['time'][xloc1], color='green', linestyle='--')
    plt.axvline(df['time'][xloc2], color='red', linestyle='--')
    plt.axvspan(df['time'][xloc1], df['time'][xloc2], facecolor='gray', alpha=.25)
    plt.axvline(df['time'][xlocA], color='#90EE90', linestyle=':')
    plt.axvline(df['time'][xlocB], color='#FF7276', linestyle=':')
    plt.axvspan(df['time'][xlocA], df['time'][xlocB], facecolor='gray', alpha=.25)

    # fig = plt.gcf()
    plot_MC(fig)

In [16]:
def plot_MC(fig):
    # Creating the Tkinter canvas containing the Matplotlib figure 
    plt.close()
    canvas = FigureCanvasTkAgg(fig, master=frame_MC)   
    canvas.draw() 
  
    # Placing the canvas on the Tkinter window 
    canvas.get_tk_widget().grid(row=0, column=0) 
  
    # Creating the Matplotlib toolbar 
    toolbar = NavigationToolbar2Tk(canvas, frame_MC, pack_toolbar=False) 
    toolbar.update() 
    toolbar.grid(row=1, column=0)

In [17]:
def updateVLine1(frame):
    vline1.set_xdata(frame)
    return vline1

In [18]:
def updateVLine2(frame):
    vline2.set_xdata(frame)
    return vline2

In [19]:
def updateVLineA(frame):
    vlineA.set_xdata(frame)
    return vlineA

In [20]:
def updateVLineB(frame):
    vlineB.set_xdata(frame)
    return vlineB

In [21]:
def openNewWindow():
    newWindow = tk.Toplevel(window)
    newWindow.title("Temp Window")
    newWindow.geometry("800x800")
    tk.Label(newWindow, text = "Test (WIP)")

    btn = tk.Button(newWindow, 
				 text ="Click to close this window", 
				 command = newWindow.destroy)
    btn.pack()

In [22]:
def openBig5():
	newBig5 = tk.Toplevel(window)
	newBig5.title("Temp Big 5 Window")
	newBig5.geometry("1200x1000")
	tk.Label(newBig5, text = "The \'Big 5\' represents the 3 major measureables, as well as the corresponding SSA and BCMass").pack()

	btn = tk.Button(newBig5, 
				 text ="Click to close this window", 
				 command = newBig5.destroy)
	btn.pack()


    ##In the new window, plot the "big 5" measurements
	fig = Figure(figsize = (12, 8), 
                 dpi = 100) 


	#TODO: Loop the plots below as needed, for cleanliness
    
    # adding the subplots 
	plot1 = fig.add_subplot(321)
	plot2 = fig.add_subplot(323)
	plot3 = fig.add_subplot(325)
	plot4 = fig.add_subplot(322)
	plot5 = fig.add_subplot(324)

	ax1=plot1
	ax2=plot2
	ax3=plot3
	ax4=plot4
	ax5=plot5

	locator = mdates.AutoDateLocator()
	formatter = mdates.ConciseDateFormatter(locator)
	ax1.xaxis.set_major_locator(locator)
	ax1.xaxis.set_major_formatter(formatter)
	ax2.xaxis.set_major_locator(locator)
	ax2.xaxis.set_major_formatter(formatter)
	ax3.xaxis.set_major_locator(locator)
	ax3.xaxis.set_major_formatter(formatter)
	ax4.xaxis.set_major_locator(locator)
	ax4.xaxis.set_major_formatter(formatter)
	ax5.xaxis.set_major_locator(locator)
	ax5.xaxis.set_major_formatter(formatter)
    
	ax1.plot(df['time'],df['Bscat (1/Mm)'],label = 'Bscat')
	ax2.plot(df['time'],df['Babs (1/Mm)'],label = 'Babs')
	ax3.plot(df['time'],df['Bext (1/Mm)'],label = 'Bext')
	ax4.plot(df['time'],df['Single Scat Albedo'],label = 'SSA')
	ax5.plot(df['time'],df['BC Mass (ug/m3)'],label = 'BC Mass')

	
	ax1.tick_params(axis = 'both', labelsize = 'small',labelrotation=20)
	ax2.tick_params(axis = 'both', labelsize = 'small',labelrotation=20)
	ax3.tick_params(axis = 'both', labelsize = 'small',labelrotation=20)
	ax4.tick_params(axis = 'both', labelsize = 'small',labelrotation=20)
	ax5.tick_params(axis = 'both', labelsize = 'small',labelrotation=20)


	ax1.set_ylabel("Bscat")
	ax2.set_ylabel("Babs")
	ax3.set_ylabel("Bext")
	ax4.set_ylabel("SSA")
	ax5.set_ylabel("BC Mass")


    # containing the Matplotlib figure 
	canvas = FigureCanvasTkAgg(fig, 
                               master = newBig5)   
	canvas.draw() 
  
    # placing the canvas on the Tkinter window 
	canvas.get_tk_widget().pack()
  
    # creating the Matplotlib toolbar 
	toolbar = NavigationToolbar2Tk(canvas, 
                                   newBig5, pack_toolbar=False) 
	toolbar.update() 
	toolbar.pack()


In [23]:
def open4x():
	new4x = tk.Toplevel(window)
	new4x.title("Temp 4x Sanity Check Window")
	new4x.geometry("1200x1000")
	tk.Label(new4x, text = "Test (WIP)").pack()
    
	btn = tk.Button(new4x, 
				 text = "Click to close this window", 
				 command = new4x.destroy)
	btn.pack()

    ##In the new window, plot the "4x4" measurements
	fig = Figure(figsize = (12, 8), 
                 dpi = 100) 


	#TODO: Loop the plots below as needed
    
    # adding the subplots 
	plot1 = fig.add_subplot(221)
	plot2 = fig.add_subplot(222)
	plot3 = fig.add_subplot(223)
	plot4 = fig.add_subplot(224)


	ax1 = plot1
	ax2 = plot2
	ax3 = plot3
	ax4 = plot4

	locator = mdates.AutoDateLocator()
	formatter = mdates.ConciseDateFormatter(locator)
	ax1.xaxis.set_major_locator(locator)
	ax1.xaxis.set_major_formatter(formatter)
	ax2.xaxis.set_major_locator(locator)
	ax2.xaxis.set_major_formatter(formatter)
	ax3.xaxis.set_major_locator(locator)
	ax3.xaxis.set_major_formatter(formatter)
	ax4.xaxis.set_major_locator(locator)
	ax4.xaxis.set_major_formatter(formatter)
    
	ax1.plot(df['time'],df['scat_raw'],label = 'Bscat RAW')
	ax2.plot(df['time'],df['Bext (1/Mm)'],label = 'Bext')
	ax3.plot(df['time'],df['Detected Laser power (W)'],label = 'Laser Power')
	ax4.plot(df['time'],df['time']) #This is a placeholder until r^2 is plotted


	
	ax1.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)
	ax2.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)
	ax3.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)
	ax4.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)



	ax1.set_ylabel("Bscat RAW")
	ax2.set_ylabel("Bext")
	ax3.set_ylabel("Laser Power")
	ax4.set_ylabel('Placeholder. "Generate calibration frame" for r^2')

    # containing the Matplotlib figure 
	canvas = FigureCanvasTkAgg(fig, 
                               master = new4x)   
	canvas.draw() 
  
    # placing the canvas on the Tkinter window 
	canvas.get_tk_widget().pack()
  
    # creating the Matplotlib toolbar 
	toolbar = NavigationToolbar2Tk(canvas, 
                                   new4x, pack_toolbar=False) 
	toolbar.update() 
	toolbar.pack()

In [24]:
def openFilterCompare():
	newCompare = tk.Toplevel(window)
	newCompare.title("Temp Filtered comparison Window")
	newCompare.geometry("1200x1000")
	tk.Label(newCompare, text = "Test (WIP)")

	btn = tk.Button(newCompare, 
				 text = "Click to close this window", 
				 command = newCompare.destroy)
	btn.pack()


    ##In the new window, plot the "4x4" measurements
	fig = Figure(figsize = (12, 8), 
                 dpi = 100) 


	#TODO: Loop the plots below as needed
        #Ex: for x in list(
    
    # adding the subplots 
	plot1 = fig.add_subplot(221)
	plot2 = fig.add_subplot(222)


	ax1 = plot1
	ax2 = plot2


	locator = mdates.AutoDateLocator()
	formatter = mdates.ConciseDateFormatter(locator)
	ax1.xaxis.set_major_locator(locator)
	ax1.xaxis.set_major_formatter(formatter)
	ax2.xaxis.set_major_locator(locator)
	ax2.xaxis.set_major_formatter(formatter)

    
	ax1.plot(df['time'],df['scat_raw'],label = 'Bscat RAW') #TODO: Change this 
	ax2.plot(df['time'],df['Bext (1/Mm)'],label = 'Bext') #TODO: Change this

	
	ax1.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)
	ax2.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)

    
	ax1.set_ylabel("PLACEHOLDER: FILTERED")
	ax2.set_ylabel("PLACEHOLDER: UNFILTERED")


    # containing the Matplotlib figure 
	canvas = FigureCanvasTkAgg(fig, 
                               master = newCompare)   
	canvas.draw() 
  
    # placing the canvas on the Tkinter window 
	canvas.get_tk_widget().pack()
  
    # creating the Matplotlib toolbar 
	toolbar = NavigationToolbar2Tk(canvas, 
                                   newCompare, pack_toolbar=False) 
	toolbar.update() 
	toolbar.pack()

In [25]:
def alarm_translate():
	# newCodeWindow = tk.Toplevel(window)
	# newCodeWindow.title("Alarm Code translator")
	# newCodeWindow.geometry("800x500")
	# tk.Label(newCodeWindow, text = "Test (WIP)")

	user_alarm_string = alarmTextbox.get()

	for index in range(0, len(user_alarm_string)):
        
		match user_alarm_string[index]:
			case 'r':
				writeToLog(f'{alarm_names[index]:25} RED')
			case 'y':
				writeToLog(f'{alarm_names[index]:25} YELLOW')
			case 'g':
				pass


In [ ]:

def chooseBestFit():
	newFitWindow = tk.Toplevel(window)
	newFitWindow.title("Testing line of best fit")
	newFitWindow.geometry("1000x800")
	tk.Label(newFitWindow, text = "Test (WIP)")

    
	btn = tk.Button(newFitWindow, 
				 text = "Click to close this window", 
				 command = newFitWindow.destroy)
	btn.grid(row = 0, column = 0)

	frame_top = tk.Frame(newFitWindow)
	frame_top.grid(row = 1, column = 0)
	s1 = tk.ttk.Separator(newFitWindow, orient = 'horizontal').grid(row = 2,columnspan = 9,sticky = "ew")
	frame_bottom = tk.ttk.Labelframe(newFitWindow, text = 'Fit numbers:')
	frame_bottom.grid(row = 3, column = 0)

    
	fig = Figure(figsize = (10, 5), 
                 dpi = 100) 
    # adding the subplots 
	plot1 = fig.add_subplot(121)
	plot2 = fig.add_subplot(122)


	ax1 = plot1
	ax2 = plot2


	locator = mdates.AutoDateLocator()
	formatter = mdates.ConciseDateFormatter(locator)
	ax1.xaxis.set_major_locator(locator)
	ax1.xaxis.set_major_formatter(formatter)


    
	min = float(entry_min.get())
	max = float(entry_max.get())
	percent = float(entry_percent.get())


    
	r2 = 0
	slope = 0



    
	if calibvar.get() == 'Scattering':
		
		filtered_time = df['time'].iloc[xlocA:xlocB]
		filtered_dfx = df['Bscat (1/Mm)'].iloc[xlocA:xlocB]
		filtered_dfy = df['Debug Ext Calculation'].iloc[xlocA:xlocB]


		y_pct_change = filtered_dfy.pct_change() * 100
		filtered_dfy = filtered_dfy[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]
		filtered_dfx = filtered_dfx[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]
		filtered_time = filtered_time[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]
        
		mask = (filtered_dfy >= min) & (filtered_dfy <= max)
		filtered_dfx = filtered_dfx[mask]
		filtered_dfy = filtered_dfy[mask]
		filtered_time = filtered_time[mask]
    
    
		slope, intercept, r_value, p_value, std_err = linregress(filtered_dfx, filtered_dfy)
		r2 = r_value **2
		sns.regplot(data = df, x = filtered_dfx,y = filtered_dfy, ax = ax1, marker = 'x', line_kws = dict(color = 'r'), fit_reg = True)

		plt.xticks(rotation = 20)

		ax2.scatter(filtered_time,filtered_dfy,label = 'Filtered')

    
#Repeat the graph setup above, just changing the y axis
    
	elif calibvar.get() == 'Absorbing':  

		filtered_time = df['time'].iloc[xlocA:xlocB]
		filtered_dfx = df['Babs (1/Mm)'].iloc[xlocA:xlocB]

		difference_dfy = (df['Debug Ext Calculation'].iloc[xlocA:xlocB]-df['Bscat (1/Mm)'].iloc[xlocA:xlocB])
		# filtered_dfy = difference_dfy.iloc[xlocA:xlocB]

		y_pct_change = difference_dfy.pct_change() * 100
		filtered_dfy = difference_dfy[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]
		filtered_dfx = filtered_dfx[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]
		filtered_time = filtered_time[(y_pct_change.abs() <= percent) | (y_pct_change.isna())]        

        
		mask = (filtered_dfy >= min) & (filtered_dfy <= max)
		filtered_dfx = filtered_dfx[mask]
		filtered_dfy = filtered_dfy[mask]
		filtered_time = filtered_time[mask]        
    
		combined_df = pd.concat([filtered_dfx, filtered_dfy],axis=1)
        
		slope, intercept, r_value, p_value, std_err = linregress(filtered_dfx, filtered_dfy)
		r2 = r_value **2
		sns.regplot(data = combined_df, x = filtered_dfx,y = filtered_dfy, ax = ax1, marker = 'x', line_kws = dict(color = 'r'), fit_reg = True)

		ax2.scatter(filtered_time,filtered_dfy,label = 'Filtered')

	else:
		print('did not work as expected')
	
	ax1.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)
	ax2.tick_params(axis = 'both', labelsize = 'small',labelrotation = 20)    


    #Let's rename this set of axis labels, and include a better frame representation for the data presented
	ax1.set_ylabel("Lin Reg Best Fit, given constraints")
	ax2.set_ylabel("Filtered data")

	label_r2 = tk.ttk.Label(frame_bottom, text = f"r^2: {r2}")
	label_r2.grid(row = 0, column = 0)
	label_slope = tk.ttk.Label(frame_bottom, text = f"Slope: {slope}")
	label_slope.grid(row = 1, column = 0)

    
    # containing the Matplotlib figure 
	canvas = FigureCanvasTkAgg(fig, 
                               master = frame_top)   
	canvas.draw() 
  
    # placing the canvas on the Tkinter window 
	canvas.get_tk_widget().pack()
  
    # creating the Matplotlib toolbar 
	toolbar = NavigationToolbar2Tk(canvas, 
                                   frame_top, pack_toolbar=False) 
	toolbar.update() 
	toolbar.pack()

    

In [27]:
def are_you_sure():
	"""
	Just a function that opens a message box to confirm window destruction
	"""

	if tk.messagebox.askyesno("Quit Dialog","Are you sure you want to quit the app?"):
		window.destroy()

In [28]:
def writeToLog(msg):
    numlines = int(log.index('end - 1 line').split('.')[0])
    log['state'] = 'normal'
    if numlines == 24:
        log.delete(1.0, 2.0)
    if log.index('end-1c') != '1.0':
        log.insert('end', '\n')
    log.insert('end', msg)
    log['state'] = 'disabled'

In [29]:
def update_label1(value):
    label_slider1.config(text=f"Slider Value: {value}")

In [30]:
def update_label2(value):
    label_slider2.config(text=f"Slider Value: {value}")

In [31]:
def update_labelA(value):
    label_sliderA.config(text=f"Slider Value: {value}")

In [32]:
def update_labelB(value):
    label_sliderB.config(text=f"Slider Value: {value}")

In [33]:
def slider_changed1(event):
    # print(current_value.get())
    xlocMax = len(df['time'])
    
    global xloc1 #As it stands, xloc1 is defined here to prevent bugs. Probably a better way to do this that is not global   
 
    xloc1 = int(len(df['time'])*(current_value1.get()/1000))

    if xloc1 >= xlocMax:
        xloc1 -= 1
    
    update_label1(str(df.at[xloc1,'time']))
    plot()
    
    # binPlot()

In [34]:
def slider_changed2(event):
    # print(current_value.get())
    xlocMax = len(df['time'])

    global xloc2 
    
    xloc2 = int(len(df['time'])*(current_value2.get()/1000))

    
    if xloc2 >= xlocMax:
        xloc2 -= 1
    
    update_label2(str(df.at[xloc2,'time']))
    plot()
    # updateVLine2(frame_MC)
    # print(xloc)


In [35]:
def slider_changedA(event):
    xlocMax = len(df['time'])
    
    global xlocA #As it stands, xloc1 is defined here to prevent bugs. Probably a better way to do this that is not global
    
 
    xlocA = int(len(df['time'])*(current_valueA.get()/1000))

    if xlocA >= xlocMax:
        xlocA -= 1
    
    update_labelA(str(df.at[xlocA,'time']))
    plot()


In [36]:
def slider_changedB(event):
    xlocMax = len(df['time'])
    
    global xlocB #As it stands, xloc1 is defined here to prevent bugs. Probably a better way to do this that is not global
    
 
    xlocB = int(len(df['time'])*(current_valueB.get()/1000))

    if xlocB >= xlocMax:
        xlocB -= 1
    
    update_labelB(str(df.at[xlocB,'time']))
    plot()


In [37]:
#Layout: New

#TL
frame_TL = tk.Frame(window)
load_file_button = tk.Button(frame_TL, text="Load PAX data file", command=load_file, width=30, bg='orange')
load_file_button.grid(row=0, column=0, columnspan=3)

selected = tk.StringVar()
selected.set("V1")
radio_csv = tk.Radiobutton(frame_TL, text="Load default .csv", value="V1", variable=selected)
radio_xlsx = tk.Radiobutton(frame_TL, text="Load default .xlsx", value="V2", variable=selected)
radio_csv.grid(row=1, column=0)
radio_xlsx.grid(row=1, column=1)

#TC
frame_TC = tk.Frame(window)
log = tk.Text(frame_TC, state='disabled', width=70, height=5, wrap='char')
log.grid(row=0, column=0)


#TR

frame_TR = tk.Frame(window)
myImgLogo = Image.open(resource_path("DropletLogoTemp.png"))
myImgLogo = ImageTk.PhotoImage(myImgLogo)
image_label = tk.Label(frame_TR, image=myImgLogo)
image_label.pack()

In [ ]:
#ML
list_frame=tk.Frame(window)
listbox=tk.Listbox(list_frame, selectmode='multiple', height=30, width=30)
listbox.bind('<<ListboxSelect>>', plot)
scrollbar=tk.Scrollbar(list_frame, orient="vertical")
listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command = listbox.yview)

#MC
frame_MC = tk.Frame(window)


In [ ]:
#MR
container_MR = CollapsibleFrame(window, title='Important Numbers and Calibration Settings:')
container_MR.grid(row=2,column=4)

frame_MR = tk.ttk.Labelframe(container_MR.sub_frame, text='Important Numbers and Calibration Settings:')

sMR = tk.ttk.Separator(frame_MR, orient='vertical').grid(column=1,rowspan=9,sticky="ns")

calibvar = tk.StringVar()
calibration_select = tk.ttk.Combobox(frame_MR, textvariable=calibvar)
calibration_select['values'] = ('Scattering','Absorbing')
calibration_select.grid(row = 0, column = 0)

label_min = tk.ttk.Label(frame_MR, text = "Min:")
label_min.grid(row = 1, column = 0)
label_max = tk.ttk.Label(frame_MR, text = "Max:")
label_max.grid(row = 2, column = 0)
#ToDo: Check escape character
label_percent = tk.ttk.Label(frame_MR, text = "% change:")
label_percent.grid(row = 3, column = 0)

entry_min = tk.Entry(frame_MR, width = 20)
entry_min.grid(row = 1, column = 2)
entry_max = tk.Entry(frame_MR, width = 20)
entry_max.grid(row = 2, column = 2)
entry_percent = tk.Entry(frame_MR, width = 20)
entry_percent.grid(row = 3, column = 2)

calibStarter = tk.Button(frame_MR, 
             text = "Generate calibration frame", 
             command = chooseBestFit, bg = 'light blue')
calibStarter.grid(row = 4, column = 2)


label_span1 = tk.Label(frame_MR, text = "I0 selection:")
label_span1.grid(row = 5, column = 0)

current_value1 = tk.DoubleVar()

slider1 = tk.ttk.Scale(
    frame_MR,
    from_ = 0,
    to = 1000,
    orient = 'horizontal',
    variable = current_value1,
    command = slider_changed1
).grid(row = 6,column = 0)

current_value1.set(0) #Arbitrary number to start at

label_slider1 = tk.Label(frame_MR, text = "Slider is at " + str(current_value1.get()), fg = 'green')
label_slider1.grid(row = 6, column = 2)

current_value2 = tk.DoubleVar()

slider2 = tk.ttk.Scale(
    frame_MR,
    from_ = 0,
    to = 1000,
    orient = 'horizontal',
    variable = current_value2,
    command = slider_changed2
).grid(row = 7,column = 0)


label_slider2 = tk.Label(frame_MR, text = "Slider is at " + str(current_value2.get()), fg = 'red')
label_slider2.grid(row = 7, column = 2)

current_valueA = tk.DoubleVar()

label_spanA = tk.Label(frame_MR, text = "Calib. Region:")
label_spanA.grid(row = 8, column = 0)


sliderA = tk.ttk.Scale(
    frame_MR,
    from_ = 0,
    to = 1000,
    orient = 'horizontal',
    variable = current_valueA,
    command = slider_changedA
).grid(row = 9,column = 0)

current_valueA.set(0) #Arbitrary number to start at

current_valueB = tk.DoubleVar()

sliderB = tk.ttk.Scale(
    frame_MR,
    from_ = 0,
    to = 1000,
    orient = 'horizontal',
    variable = current_valueB,
    command = slider_changedB
).grid(row = 10,column = 0)

current_valueB.set(0) #Arbitrary number to start at


label_sliderA = tk.Label(frame_MR, text = "Slider is at " + str(current_valueA.get()), fg = '#90EE90')
label_sliderA.grid(row = 9, column = 2)

label_sliderB = tk.Label(frame_MR, text = "Slider is at " + str(current_valueB.get()), fg = '#FF7276')
label_sliderB.grid(row = 10, column = 2)


# plotStarter = tk.Button(frame_MR, 
#              text ="Replot", 
#              command = lambda: openNewWindow(), bg = 'light blue')
# plotStarter.grid(row=10, column=2)

alarmTextbox = tk.ttk.Entry(frame_MR)
alarmTextbox.grid(row = 12, column = 0)

translateButton = tk.Button(frame_MR, 
             text = "Translate Alarm", 
             command = lambda: alarm_translate(), bg = 'light blue')
translateButton.grid(row = 12, column = 2)

#ToDo: Add a filter/unfilter option
#Todo: include measurement parameters for the data snippet demarkation


In [40]:
#BL
frame_BL = tk.Frame(window)
button_quit = tk.Button(master = frame_BL, text = "Quit", command = are_you_sure, bg = '#FF2222')
button_clear = tk.Button(master = frame_BL, text = "Clear Selection", command = lambda: listbox.selection_clear(0,'end'), bg = 'light green')
testTextButton = tk.Button(master = frame_BL, text = "Display filename", command = lambda: writeToLog(file_path), width = 15, bg = 'light yellow')

In [ ]:
#BC
frame_BC = tk.Frame(window)
file_path = "temp"

PrepBtn1 = tk.Button(frame_BC, 
             text ="Big 5 measurables", 
             command = openBig5, bg = 'light yellow')
PrepBtn1.grid(row = 0, column = 0)

PrepBtn2 = tk.Button(frame_BC, 
             text ="4x sanity check", 
             command = open4x, bg = 'light yellow')
PrepBtn2.grid(row = 0, column = 1)

# PrepBtn3 = tk.Button(frame_BC, 
#              text = "Filtered Data - Bscat", 
#              command = openFilterCompare, bg = 'light yellow')
# PrepBtn3.grid(row = 0, column = 1)
# PrepBtn4 = tk.Button(frame_BC, 
#              text = "Filtered Data - Babs", 
#              command = openFilterCompare, bg = 'light yellow')
# PrepBtn4.grid(row = 1, column = 1)
# PrepBtn5 = tk.Button(frame_BC, 
#              text = "Some other 5th thing", 
#              command = openFilterCompare, bg = 'light yellow')
# PrepBtn5.grid(row = 1, column = 1)

In [42]:
#BR + Separators
frame_BR = tk.Frame(window)
label_progress = tk.ttk.Label(frame_BR, text = "(This progress bar will complete when\na file is loaded and column chosen)")
label_progress.grid(row = 0, column = 0)
pb = ttk.Progressbar(frame_BR, orient = 'horizontal', mode = 'determinate', length = 250)
pb.grid(column = 0, row = 1, padx = 10, pady = 20)


#Other(Separators)

##Question: Playing around with position still - is there any way to have separators go through each other? -(Check out "weight" option)

s1 = tk.ttk.Separator(window, orient = 'horizontal').grid(row = 1, columnspan = 9,sticky = "ew")
s4 = tk.ttk.Separator(window, orient = 'vertical').grid(column = 3,rowspan = 9,sticky = "ns")
s2 = tk.ttk.Separator(window, orient = 'horizontal').grid(row = 3, columnspan = 9,sticky = "ew")
s3 = tk.ttk.Separator(window, orient = 'vertical').grid(column = 1,rowspan = 9,sticky = "ns")

#Other(Menu)
main_menu = tk.Menu(window)
window.config(menu = main_menu)

sub_menu_file = tk.Menu(main_menu, tearoff = 0)
main_menu.add_cascade(label = "File", menu = sub_menu_file)
sub_menu_file.add_command(label = "Exit", command = are_you_sure)

#Placements (grid)

frame_TL.grid(row = 0, column = 0)
frame_TC.grid(row = 0, column = 2)
frame_TR.grid(row = 0, column = 4)
frame_MC.grid(row = 2, column = 2)
frame_MR.grid(row = 2, column = 4)
frame_BL.grid(row = 4, column = 0)
frame_BC.grid(row = 4, column = 2)
frame_BR.grid(row = 4, column = 4)
list_frame.grid(row = 2, column = 0)
button_quit.grid(row = 2, column = 0)
#button_test.grid(row=4, column=0)
button_clear.grid(row = 0, column = 0)
testTextButton.grid(row = 1, column = 0)
listbox.pack(side = "left", fill = "y")
scrollbar.pack(side = "right", fill = "y")

In [ ]:
#run the gui
window.mainloop()

#application.__init__ ?Is this the correct format?

KeyboardInterrupt: 

: 